## No Manifesto Test Run
This is a useful test run of scoring the parties without any manifesto text, as a baseline for what the LLM knows about the party and it's positions. 
If this work should be pursued it can be added to the modules. 

In [1]:
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

from llm_political_analysis.modules.prompts import policy_scales

In [2]:
def score_party(party, year, issue):
    '''
    Function to score a party's policy position on a given issue in a given year
    
    Parameters:
        party (str): The name of the party.
        year (int): The year for which the policy position is being scored.
        issue (str): The issue for which the policy position is being scored.
    
    Returns:
        dict: A dictionary containing the party, year, issue, score, and error_message (if any).
            - party (str): The name of the party.
            - year (int): The year for which the policy position is being scored.
            - issue (str): The issue for which the policy position is being scored.
            - score (str): The policy position score.
            - error_message (str): Any error message encountered during the scoring process, or None if no error occurred.
    '''
    prompt = [SystemMessage(content=new_prompts[issue]), HumanMessage(content=f'What is the policy position of the {party} in {year} on {issue}?')]
    llm = ChatOpenAI(temperature=0, max_tokens=150, model_name='gpt-4o', max_retries=7)
    response_dict = {'party': party, 'year': year, 'issue': issue}
    try: 
        response = llm.invoke(prompt)
        score = response.content.strip()
        response_dict['score'] = score
        response_dict['error_message'] = None
    except Exception as e:
        response_dict['score'] = 'NA'
        response_dict['error_message'] = e
    return response_dict

In [3]:
policy_scales

{'taxation': 'You are conducting research on the policy positions that European parties in parliamentary democracies take in their political manifestos.  Political manifestos are documents parties produce to explain their policy positions to voters in an election.  For the following text of a party manifesto, please classify the party position on the tradeoff between higher spending on public services and lower taxation, using the following seven point scale: \n\n1.\tHigh Public Spending, High Taxation: The party strongly advocates for high levels of public spending on public services, accompanied by high taxation to fund this expenditure. This position prioritizes extensive public services and welfare programs, emphasizing the role of the government in providing these services.\n2.\tModerately High Public Spending, High Taxation: The party supports relatively high levels of public spending and is comfortable with high taxation to support this. While not as extreme as position 1, it st

In [4]:
trimmed_scales = [scale.split(',')[1:] for scale in policy_scales.values()]

In [5]:
trimmed_scales

[[' please classify the party position on the tradeoff between higher spending on public services and lower taxation',
  ' using the following seven point scale: \n\n1.\tHigh Public Spending',
  ' High Taxation: The party strongly advocates for high levels of public spending on public services',
  ' accompanied by high taxation to fund this expenditure. This position prioritizes extensive public services and welfare programs',
  ' emphasizing the role of the government in providing these services.\n2.\tModerately High Public Spending',
  ' High Taxation: The party supports relatively high levels of public spending and is comfortable with high taxation to support this. While not as extreme as position 1',
  ' it still prioritizes significant public investment and welfare services',
  ' albeit with some consideration for tax efficiency.\n3.\tModerate Public Spending',
  ' Moderate to High Taxation: The party advocates for a balanced approach with moderately high public spending and corre

In [6]:
new_prompts = {key: 'You are an expert social scientist with a PhD in political science.  Think carefully about your answer. You are conducting research on the policy positions of the following European parties in parliamentary democracies in a specific year'+','.join(value) for key, value in zip(policy_scales.keys(), trimmed_scales)}

In [7]:
new_prompts

{'taxation': 'You are an expert social scientist with a PhD in political science.  Think carefully about your answer. You are conducting research on the policy positions of the following European parties in parliamentary democracies in a specific year please classify the party position on the tradeoff between higher spending on public services and lower taxation, using the following seven point scale: \n\n1.\tHigh Public Spending, High Taxation: The party strongly advocates for high levels of public spending on public services, accompanied by high taxation to fund this expenditure. This position prioritizes extensive public services and welfare programs, emphasizing the role of the government in providing these services.\n2.\tModerately High Public Spending, High Taxation: The party supports relatively high levels of public spending and is comfortable with high taxation to support this. While not as extreme as position 1, it still prioritizes significant public investment and welfare s

In [8]:
issues = policy_scales.keys()
party_years = [('National Coalition Party KOK in Finland', '2019'),
               ('Freedom Party of Austria', '2006'),
               ('Groen Party (FPO) in Belgium', '2010'),
               ('Attack Party in Bulgaria', '2014'),
               ('Communist Party of Bohemia and Moravia (KSCM) in Czech Republic', '2010'),
               ("People's Party for Freedom and Democracy (VVD) in the Netherlands", '2006'),
               ('Direction – Social Democracy in Slovakia', '2006'),
               ('Green Party in UK', '2019')] 

In [9]:
results = []
for party, year in party_years:
    for issue in issues:
        results.append(score_party(party, year, issue))

In [10]:
results_df = pd.DataFrame(results)

In [11]:
results_df

,party,year,issue,score,error_message
0,National Coalition Party KOK in Finland,2019,taxation,5,None
1,National Coalition Party KOK in Finland,2019,lifestyle,3,None
2,National Coalition Party KOK in Finland,2019,immigration,4,None
3,National Coalition Party KOK in Finland,2019,environment,4,None
4,National Coalition Party KOK in Finland,2019,european_union,1,None
5,National Coalition Party KOK in Finland,2019,decentralization,4,None
6,Freedom Party of Austria,2006,taxation,6,None
7,Freedom Party of Austria,2006,lifestyle,6,None
8,Freedom Party of Austria,2006,immigration,7,None
9,Freedom Party of Austria,2006,environment,6,None


In [62]:
results_df.to_excel('../data/results/party_policy_scores.xlsx', index=False)